In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy
import matplotlib.pyplot as plt

%matplotlib inline

#### SVMのスクラッチ実装(二次計画法で解く)

In [215]:
import cvxopt
from cvxopt import matrix

N = 100
#X = np.random.randn(100, 2)
cls1 = []
cls2 = []

mean1 = [-1, 2]
mean2 = [1, -1]
cov = [[1.0,0.8], [0.8, 1.0]]
    
cls1.extend(np.random.multivariate_normal(mean1, cov, int(N/2)))
cls2.extend(np.random.multivariate_normal(mean2, cov, int(N/2)))
    
# データ行列Xを作成
X = np.vstack((cls1, cls2))
T =[]
for i in range(int(N/2)):
    T.append(1.0)
    
for i in range(int(N/2)):
    T.append(-1.0)
T = np.array(T)

In [221]:
class SVM():
    
    def fit(self, X ,y):
        
        '''
         二次計画法を定義して解く
        '''
        ##行列作成
        m = np.zeros((X.shape[0] , X.shape[0]))
        for i in range(X.shape[0]):
            for j in range(X.shape[0]):
                m[i , j] = T[i] * T[j] * np.dot(X[i] , X[j])
        
         ##作成された行列を30x30へ
        P = matrix(m)
        ##残りの行列を定義
        q = matrix(np.array([-1.0 for i in range(X.shape[0])]))
        G = matrix(-1.0*np.eye(X.shape[0]))
        h = matrix(np.array([.0 for i in range(X.shape[0])]))
        A = matrix(T , (1,T.shape[0]))
        b = matrix(0.0)
        
        #self.P = P
        
        ##二次計画法で解く。
        sol=cvxopt.solvers.qp(P,q,G,h,A,b)
        a = np.array(sol['x'])
        #self.a = a
        
        # サポートベクトルのインデックスを抽出
        S = []
        for i in range(len(a)):
            if a[i] < 0.00001: continue
            S.append(i)
    
        # wを計算
        self.w = np.zeros(2)
        for n in S:
            self.w += a[n] * t[n] * X[n]
    
        # w0を計算
        sums = 0
        for n in S:
            temp = 0
            for m in S:
                temp += a[m] * t[m] * kernel(X[n], X[m])
            sums += (t[n] - temp)
        self.w0 = sums / len(S)
    

        return self
        
    def predict(self , X):
        pred_list =  np.sign(self.w0 + np.dot(X , self.w))
        ok = []
        for i in range(len(X)):
            if T[i] == pred_list[i]:
                ok.append(1)
                
            else:
                ok.append(0)
                
        acc_SVM = np.sum(ok)/len(ok)
        return print('Accuracy is {}'.format(acc_SVM))
            
        pass 
            
if __name__ =='__main__':
    model = SVM()
    model.fit(X ,T)
    print(model.predict(X))

     pcost       dcost       gap    pres   dres
 0: -6.9546e+00 -1.2578e+01  3e+02  2e+01  2e+00
 1: -7.9314e+00 -3.8777e+00  5e+01  3e+00  3e-01
 2: -2.5665e+00 -1.5334e+00  8e+00  4e-01  5e-02
 3: -1.0320e+00 -1.2948e+00  3e-01  4e-04  5e-05
 4: -1.2290e+00 -1.2336e+00  5e-03  8e-06  9e-07
 5: -1.2325e+00 -1.2326e+00  5e-05  8e-08  9e-09
 6: -1.2326e+00 -1.2326e+00  5e-07  8e-10  9e-11
Optimal solution found.
Accuracy is 1.0
None


#### NNのスクラッチ実装

In [170]:
class NN():
    '''
    3層のNNにおいて活性化関数をシグモイド関数として実装
    '''
    
    def __init__(self , input_node , hidden_node , output_node, lr_rate):
        '''
        各レイヤーのノード数、重み、学習率、活性化関数の設定
        '''
        
        self.input_node  = input_node
        self.hidden_node = hidden_node
        self.output_node  = output_node
        
        ##各レイヤー間の重みの設定
        ##各重みを(output , input)のshapeで定義しているのは、error更新の際に重みの行列を転置して、各ノードの誤差計算をするため
        self.between_input_hidden = np.random.normal(.0 , np.sqrt(2/self.input_node) ,(self.hidden_node , self.input_node) )
        self.between_hidden_output = np.random.normal( .0 , np.sqrt(2/self.hidden_node)  ,(self.output_node , self.hidden_node))
        
        ##学習率の設定
        self.lr_rate = lr_rate
        
        ##活性化関数の設定
        self.activation_func = lambda x: 1/ (1 + np.exp(-x))
        
        
        
    def train(self, train_data , target):
        '''
        入力、隠れ層、出力層、各レイヤー間の重みの更新
        '''
        ##入力の行列変換
        inputs = np.array(train_data , ndmin=2).T
        targets = np.array(target , ndmin=2).T
        
        ##隠れ層へのinputの計算
        hidden_inputs = np.dot(self.between_input_hidden , inputs)
        ##隠れ層のactivate function
        hidden_outputs = self.activation_func(hidden_inputs)
        
        ##出力層へのinputの計算
        final_inputs = np.dot(self.between_hidden_output , hidden_outputs)
        ##出力層のactivate function
        final_outputs = self.activation_func(final_inputs)
        
        ##誤差計算
        output_errors = targets  - final_outputs
        
        ##誤差から出力層と隠れ層の重みを更新
        hidden_errors  = np.dot(self.between_hidden_output.T  , output_errors)
        self.between_hidden_output += self.lr_rate*np.dot((output_errors*final_outputs*(1.0 - final_outputs)) , hidden_outputs.T)
        
        ##誤差から隠れ層と入力層の重みを更新
        self.between_input_hidden += self.lr_rate*np.dot((hidden_errors*hidden_outputs*(1.0 - hidden_outputs)) , inputs.T)
        
        
        
    def test(self , test_data):
        '''
        各レイヤーからの出力値を計算して、確率値の出力を得る。そして、その最大値のindexを分類とする
        '''
       
        ##入力の行列変換
        inputs = np.array(test_data  ,ndmin=2).T
        
        ##隠れ層への入力の計算
        hidden_inputs = np.dot(self.between_input_hidden , inputs)
        ##隠れ層のactivate function
        hidden_outputs = self.activation_func(hidden_inputs)
        
        ##出力層へのinputの計算
        final_inputs = np.dot(self.between_hidden_output , hidden_outputs)
        ##出力層のactivate function
        final_outputs = self.activation_func(final_inputs)
    
        return final_outputs
    
    
if __name__ =='__main__':
    mnist_train = np.array(pd.read_csv('/Users/shintaro/Downloads/mnist-in-csv/mnist_train.csv', engine='python'))
    mnist_test = np.array(pd.read_csv('/Users/shintaro/Downloads/mnist-in-csv/mnist_test.csv', engine='python'))
    input_node=784
    hidden_node=100
    output_node=10
    lr_rate=0.3

    model = NN(input_node , hidden_node , output_node , lr_rate)

    
    
    '''ここから下は別ファイルで実行'''
    epochs = 5

    '''
    データを最大値の255で割り、0~1の範囲にする
    '''

    for e in range(epochs):
        for data in train_data:
            input_data = data[1:]/255
            target = np.zeros(output_node)
            target[data[0]] = 1.0
            model.train(input_data , target)
    


    ok =[]
    for data2 in test_data:
        input_data = data2[1:]/255
        target = data2[0]
        output = model.test(input_data)
        pred_label = np.argmax(output)
        if target == pred_label:
            ok.append(1)
        else:
            ok.append(0)
        
    acc = sum(ok)/len(ok)
    print('Accuracy is {}'.format(acc))
    

Accuracy is 0.9643
